In [2]:
import numpy as np
import cv2
import glob
from tqdm import tqdm
import pandas as pd


chessboard_size = (7, 5)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space

imgpoints = [] # 2d points in image plane.

# cap = cv2.VideoCapture('chessboard.mp4')
# nb_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# nb_frames = 10
# img = None
# for i in tqdm(range(nb_frames)):
#     ret,frame = cap.read()
#     if i == 0:
#         img = frame

#     gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
#     # Find the chess board corners
#     ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
#     # If found, add object points, image points (after refining them)
#     if ret == True:
#         #print('chessboard found!')
#         objpoints.append(objp)
#         corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
#         imgpoints.append(corners2)
#         #img = cv2.drawChessboardCorners(img, chessboard_size, corners2,ret)
# #else:
# #   print('chessboard not found...')
# cap.release()
# print('cap.release()')   

images = glob.glob('small_chessboard_images/*.jpg') #glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    dims = img.shape
    height = dims[0]
    width = dims[1]
    channels = dims[2]
    # cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    # cv2.resizeWindow('image', width,height)
    # cv2.imshow('image',img)
    # cv2.waitKey(0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to a gray scale image
    # cv2.namedWindow('gray', cv2.WINDOW_NORMAL)
    # cv2.resizeWindow('gray', width,height)
    # cv2.imshow('gray',gray)
    # cv2.waitKey(0)
    
    #hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  
    #black = cv2.inRange(hsv, np.array([0, 0, 0]),  np.array([180, 255, 30]))
    # cv2.namedWindow('black', cv2.WINDOW_NORMAL)
    # cv2.resizeWindow('black', width,height)
    # cv2.imshow('black',black)
    # cv2.waitKey(0)
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        print('chessboard found!')
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, chessboard_size, corners2,ret)
        # cv2.namedWindow('chessboard', cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('chessboard', width,height)
        # cv2.imshow('chessboard',img)
        # cv2.waitKey(0)
    else:
        print('no chessboard found...')

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
img = cv2.imread('small-test.png')
h,  w = img.shape[:2]
newcameramtx, roi= cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imshow('dst',dst)
cv2.imwrite('calibresult.png',dst)

# mean_error = 0
# for i in range(len(objpoints)):
#     imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
#     error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
#     mean_error += error

# print ("total error: ", mean_error/len(objpoints))
cv2.waitKey(0)
cv2.destroyAllWindows()

chessboard found!
chessboard found!
chessboard found!
chessboard found!
chessboard found!
no chessboard found...
chessboard found!


In [ ]:

import time
import math

marked_image = dst.copy()
cv2.imshow('marked_image',marked_image)
cv2.waitKey(0)
    
def getperpcoord(ax, ay, bx, by, startLength, endLength):
    vx = bx-ax
    vx_offset = vx/2.0
    vy = by-ay
    vy_offset = vy/2.0
    print(str(vx)+" "+str(vy))
   # if(vx == 0 or vy == 0):
   #     return 0, 0, 0, 0
    mag = math.sqrt(vx*vx + vy*vy)
    vx = vx / mag
    vy = vy / mag
    temp = vx
    vx = 0-vy
    vy = temp
    cx = bx + (abs(bx-ax)/2.0) + vx * endLength
    cy = by + vy * endLength
    dx = bx + (abs(bx-ax)/2.0)  + vx * startLength #- vx * length
    dy = by + vy * startLength
    return int(cx), int(cy), int(dx), int(dy)



def getMarkersCenters(src_image):
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_100)
    parameters =  cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)
    markersCorners, markerIds, rejectedCandidates = detector.detectMarkers(src_image)
    markersCenters = []
    markerSideMillimeters = 47.0 #mm
    ratioPixelToMillimeters = 0
    ratioMillimetersToPixels = 0
    for markerCorners in markersCorners:
        markerAreaPixels = cv2.contourArea(markerCorners)
        markerSidePixels = math.sqrt(markerAreaPixels)
        ratioPixelToMillimeters += markerSidePixels/markerSideMillimeters
        ratioMillimetersToPixels += markerSideMillimeters/markerSidePixels
        M = cv2.moments(markerCorners)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            markersCenters.append((cx, cy))
    if len(markersCorners):
        ratioPixelToMillimeters /= len(markersCorners)
        ratioMillimetersToPixels /= len(markersCorners)
    return markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels

def loopGetMarkersCenters(src_image, nb_markers):
    #look for markers
    valueWhite = 0
    markersFound = False
    adjusted_image = None #cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)
    markersCenters = []
    ratioPixelToMillimeters = 0
    ratioMillimetersToPixels = 0
    adjusted_src_image = None
    while not markersFound:
        adjusted_src_image = src_image.copy()
        lower_white = np.array([valueWhite,valueWhite,valueWhite])
        upper_white = np.array([255,255,255])
        adjusted_src_mask = cv2.inRange(adjusted_src_image, lower_white, upper_white)
        adjusted_src_image = cv2.bitwise_and(adjusted_src_image,adjusted_src_image, mask=adjusted_src_mask)
        markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels = getMarkersCenters(adjusted_src_image)
        markersFound = (len(markersCenters) == nb_markers)
        valueWhite += 1
        if valueWhite > 255:
            return [], 0, 0
    return markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels

# define the alpha and beta
alpha = 1.5 # Contrast control
beta = 10 # Brightness control
# call convertScaleAbs function
# look for markers

adjusted_image = dst.copy()
nbMarkers = 2
markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels = loopGetMarkersCenters(adjusted_image, nbMarkers)
assert len(markersCenters) == nbMarkers

print('ratioPixelToMillimeters: %f' %ratioPixelToMillimeters)
print('ratioMillimetersToPixels: %f' %ratioMillimetersToPixels)
cv2.namedWindow('adjusted_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('adjusted_image', adjusted_image.shape[1], adjusted_image.shape[0])
cv2.imshow('adjusted_image',adjusted_image)
cv2.waitKey(0)

groundToWoodPlateMillimeters = 20
woodToDisplayStartMillimeters = 130
DisplayStartToDisplayEndMillimeters = 500
groundToMarkersMillimeters = 60 #mm 60 -> middle marker to ground
groundToDisplayStartMillimeters = (groundToWoodPlateMillimeters + woodToDisplayStartMillimeters) - groundToMarkersMillimeters
groundToDisplayEndMillimeters = groundToDisplayStartMillimeters + DisplayStartToDisplayEndMillimeters
groundToDisplayStartPixels = groundToDisplayStartMillimeters * ratioPixelToMillimeters
groundToDisplayEndPixels = groundToDisplayEndMillimeters * ratioPixelToMillimeters
print(groundToDisplayStartPixels, groundToDisplayEndPixels)

a = markersCenters[0] if markersCenters[0][0] >  markersCenters[1][0] else markersCenters[1]
b = markersCenters[0] if markersCenters[0][0] <  markersCenters[1][0] else markersCenters[1]

cx, cy, dx, dy = getperpcoord(
    a[0],
    a[1], 
    b[0], 
    b[1], 
    groundToDisplayStartPixels,
    groundToDisplayEndPixels,
    ) 

c = (cx, cy) if cy > dy else (dx, dy)
d = (dx, dy) if dy < cy else (cx, cy)

cv2.circle(marked_image, markersCenters[0], 7, (0, 0, 255), -1)
cv2.circle(marked_image, markersCenters[1], 7, (0, 0, 255), -1)
cv2.line(marked_image, markersCenters[0], markersCenters[1], (0, 0, 255),  1)
cv2.line(marked_image, c, d, (255, 0, 0),  10)
cv2.namedWindow('marked_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('marked_image', marked_image.shape[1], marked_image.shape[0])
cv2.imshow('marked_image',marked_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ratioPixelToMillimeters: 0.992472
ratioMillimetersToPixels: 1.007646
89.32249431938027 585.5585738714929
-345 4


In [ ]:

import time
import math
from PIL import Image, ImageFilter

rotated_image = marked_image.copy()


d_to_c_error = d[0]-c[0]
u = np.array([0, d[1] - c[1]])
v = np.array([d_to_c_error, d[1] - c[1]])

def angles(u, v): 
  #using the arccos function from numpy
  return np.arccos(u.dot(v)/(np.linalg.norm(u)*np.linalg.norm(v)))



def angles(u, v): 
  #using the arccos function from numpy
  return np.arccos(u.dot(v)/(np.linalg.norm(u)*np.linalg.norm(v)))


# # rotate our image by 45 degrees around the center of the image

f_angle = angles(u,v)
angle= math.degrees(f_angle)
print("the vectors are=",u,"and",v)
print("the angle between the two vectors is=",angle)
cv2.line(rotated_image, c, (c[0], d[1]), (255, 0, 255),  10)
cv2.line(rotated_image, c, d, (255, 0, 0),  10)
#cv2.line(rotated_truth_image, c, d, (255, 0, 0),  10)
#cv2.line(rotated_truth_image, c, (c[0], d[1]), (255, 0, 255),  10)

# # rotate our image by 45 degrees around the center of the image
M = cv2.getRotationMatrix2D(c, -angle, 1.0)
rotated_image = cv2.warpAffine(rotated_image, M, (rotated_image.shape[1], rotated_image.shape[0]))
cv2.namedWindow('rotated_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('rotated_image', rotated_image.shape[1], rotated_image.shape[0])
cv2.imshow('rotated_image',rotated_image)
cv2.waitKey(0)

MIN_HOLE_RADIUS_PIXELS = 189/2.0  * ratioPixelToMillimeters # 205.782cm

cropped_image = rotated_image.copy()
y_offset = 0
cropped_image = cropped_image[min(cy, dy)- y_offset :max(cy, dy) + 1 + y_offset, 0:(max(cx, dx)) + 1] 
cropped_dims = cropped_image.shape
cropped_height = cropped_dims[0]
print('cropped_height:%d' %cropped_height)
cropped_width = cropped_dims[1]
print('cropped_width:%d' %cropped_width)
# filtered_image = filtered_image & mask_rgb
cv2.imshow('cropped_image',cropped_image)
cv2.waitKey(0)

cropped_image = cropped_image[::, 0:int(cropped_image.shape[1] - MIN_HOLE_RADIUS_PIXELS)]
cv2.line(cropped_image, (cx, cy), (dx, dy), (0, 0, 255),  1)
# filtered_image = filtered_image & mask_rgb
cv2.imshow('cropped_image',cropped_image)
cv2.waitKey(0)

# kernel = np.ones((5,5), np.uint8)
# close_image = cv2.morphologyEx(cropped_image, cv2.MORPH_CLOSE, kernel)
# cv2.imshow('close_image',close_image)
# cv2.waitKey(0)

# loop until 1 contour
contours = None
hierarchy = None
single_contour_reached = False
masked_image = None
blur_factor = 3
blur_step = 4
kernel = np.ones((5,5),np.uint8)
transformed_image = cropped_image.copy()
it = 1
while not single_contour_reached:
    #transformed_image = cv2.GaussianBlur(transformed_image, (3,blur_factor), 0) 
    transformed_image = cv2.morphologyEx(transformed_image, cv2.MORPH_CLOSE, kernel, iterations=it)
    blue_image, green_image, red_image = cv2.split(transformed_image)  # extracting red channel
    rbin, redish_image = cv2.threshold(red_image, 55, 255, cv2.THRESH_BINARY)  # thresholding
    cv2.imshow('redish_image',redish_image)
    cv2.waitKey(0)

    # mask = cv2.inRange(masked_image,(0,0,0), (255,255,255))
    # mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    # masked_image = masked_image & mask_rgb
    contours, hierarchy = cv2.findContours(redish_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # # cv2.imshow('Contour image',frame)
    print('nbcontours: %d' %len(contours))
    if len(contours) == 1:
        single_contour_reached = True
    elif len(contours) == 2:
        contourArea0 = cv2.contourArea(contours[0])
        contourArea1 = cv2.contourArea(contours[1])
        if contourArea0 > contourArea1:
            contours = contours[::1]
        else:
            contours = contours[1::]
        single_contour_reached = True
    else:
        blur_factor += blur_step
        it = it + 1
contourned_image = np.zeros_like(cropped_image.copy())
#cv2.drawContours(contourned_image, contours, -1, (255, 255, 255), cv2.FILLED) # this gives a binary mask 
cv2.fillPoly(contourned_image, pts=contours, color=(255, 255, 255))
# contourned_image = cv2.bitwise_and(contourned_mask, contourned_image, )
#cv2.drawContours(contourned_mask, contours, -1, (0,255,0), 1)
cv2.imshow('contourned_image',contourned_image)
cv2.waitKey(0)

cv2.destroyAllWindows()

the vectors are= [   0 -496] and [  -6 -496]
the angle between the two vectors is= 0.6930603027111886
cropped_height:497
cropped_width:945
nbcontours: 1


In [ ]:


curve_image = contourned_image.copy()
curve_dims = curve_image.shape
curve_height = curve_dims[0]
print('curve_height:%d' %curve_height)
curve_width = curve_dims[1]
print('curve_width:%d' %curve_width)

coords = []
xMillimeters = []
yMillimeters = []
for y in range(curve_height):
    for x in range(curve_width):
        if curve_image[y, x, 0] == 255 and curve_image[y, x, 1] == 255 and curve_image[y, x, 2] == 255:
            coords.append((x, y))
            xMillimeter = (curve_width - x) * ratioMillimetersToPixels
            xMillimeters.append(xMillimeter)
            yMillimeter = (curve_height - y) * ratioMillimetersToPixels
            yMillimeters.append(yMillimeter)
            #cv2.circle(curve_image, (x, y), 2, (255, 0, 0), -1)
            break

DfMillimeters = pd.DataFrame(columns=['x', 'y'], data={'x':xMillimeters,'y':yMillimeters})
DfMillimeters.to_json('observed.json')

points = np.array(coords)
pts = points.reshape(-1,1,2) # now shape [3,1,2]
cv2.polylines(curve_image, [pts], False, (0, 0, 255), 5)
cv2.imshow("curve_image",curve_image)
cv2.waitKey(0)
final_image = cropped_image.copy()
cv2.polylines(final_image, [pts], False, (255, 255, 255), 1)
cv2.imshow("final_image",final_image)
cv2.waitKey(0)
print(xMillimeters)
print(yMillimeters)
       

curve_height:497
curve_width:851
[3.0229393454331843, 7.053525139344097, 9.068818036299554, 10.076464484777281, 11.084110933255008, 12.091757381732737, 12.091757381732737, 12.091757381732737, 12.091757381732737, 12.091757381732737, 12.091757381732737, 13.099403830210466, 14.107050278688194, 14.107050278688194, 14.107050278688194, 14.107050278688194, 14.107050278688194, 15.11469672716592, 16.12234317564365, 16.12234317564365, 16.12234317564365, 16.12234317564365, 16.12234317564365, 16.12234317564365, 17.12998962412138, 18.137636072599108, 18.137636072599108, 18.137636072599108, 18.137636072599108, 18.137636072599108, 19.145282521076833, 20.152928969554562, 20.152928969554562, 20.152928969554562, 20.152928969554562, 20.152928969554562, 20.152928969554562, 21.16057541803229, 22.168221866510017, 22.168221866510017, 22.168221866510017, 22.168221866510017, 22.168221866510017, 22.168221866510017, 23.175868314987746, 23.175868314987746, 24.183514763465475, 24.183514763465475, 24.18351476346547

# TRUTH

In [ ]:
truth_image = cv2.imread('truth-2.jpg')
h,  w = truth_image.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
# undistort
unwarped_truth_image = cv2.undistort(truth_image, mtx, dist, None, newcameramtx)
# crop the image
x,y,w,h = roi
unwarped_truth_image = unwarped_truth_image[y:y+h, x:x+w]
cv2.namedWindow('unwarped_truth_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('unwarped_truth_image', unwarped_truth_image.shape[1], unwarped_truth_image.shape[0])
cv2.imshow('unwarped_truth_image',unwarped_truth_image)
cv2.waitKey(0)

marked_truth_image = unwarped_truth_image.copy() #cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)
# lower_white = np.array([150,150,150])
# upper_white = np.array([255,255,255])
# mask = cv2.inRange(adjusted_truth_image, lower_white, upper_white)
# adjusted_truth_image = cv2.bitwise_and(adjusted_truth_image,adjusted_truth_image, mask= mask)
# cv2.namedWindow('adjusted_truth_image', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('adjusted_truth_image', adjusted_truth_image.shape[1], adjusted_truth_image.shape[0])
# cv2.imshow('adjusted_truth_image',adjusted_truth_image)
# cv2.waitKey(0)

nbMarkers = 2
markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels = loopGetMarkersCenters(marked_truth_image, nbMarkers)
groundToWoodPlateMillimeters = 20
woodToDisplayStartMillimeters = 120
DisplayStartToDisplayEndMillimeters = 500 + 10
groundToMarkersMillimeters = 60 #mm 60 -> middle marker to ground
groundToDisplayStartMillimeters = (groundToWoodPlateMillimeters + woodToDisplayStartMillimeters) - groundToMarkersMillimeters
groundToDisplayEndMillimeters = groundToDisplayStartMillimeters + DisplayStartToDisplayEndMillimeters
groundToDisplayStartPixels = groundToDisplayStartMillimeters * ratioPixelToMillimeters
groundToDisplayEndPixels = groundToDisplayEndMillimeters * ratioPixelToMillimeters
print(groundToDisplayStartPixels, groundToDisplayEndPixels)

a = markersCenters[0] if markersCenters[0][0] >  markersCenters[1][0] else markersCenters[1]
b = markersCenters[0] if markersCenters[0][0] <  markersCenters[1][0] else markersCenters[1]

cx, cy, dx, dy = getperpcoord(
    a[0],
    a[1], 
    b[0], 
    b[1], 
    groundToDisplayStartPixels,
    groundToDisplayEndPixels,
    ) 
c = (cx, cy) if cy > dy else (dx, dy)
d = (dx, dy) if dy < cy else (cx, cy)

cv2.circle(marked_truth_image, markersCenters[0], 7, (0, 0, 255), -1)
cv2.circle(marked_truth_image, markersCenters[1], 7, (0, 0, 255), -1)
cv2.line(marked_truth_image, markersCenters[0], markersCenters[1], (0, 0, 255),  10)
cv2.line(marked_truth_image, c, d, (255, 0, 0),  10)
cv2.namedWindow('marked_truth_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('marked_truth_image', marked_truth_image.shape[1], marked_truth_image.shape[0])
cv2.imshow('marked_truth_image',marked_truth_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
MIN_HOLE_RADIUS_PIXELS = 189/2.0  * ratioPixelToMillimeters # 205.782cm
rotated_truth_image = marked_truth_image.copy()

d_to_c_error = d[0]-c[0]
u = np.array([0, d[1] - c[1]])
v = np.array([d_to_c_error, d[1] - c[1]])
def angles(u, v): 
  #using the arccos function from numpy
  return np.arccos(u.dot(v)/(np.linalg.norm(u)*np.linalg.norm(v)))

f_angle = angles(u,v)
angle= math.degrees(f_angle)
print("the vectors are=",u,"and",v)
print("the angle between the two vectors is=",angle)
cv2.line(rotated_truth_image, c, (c[0], d[1]), (255, 0, 255),  10)
cv2.line(rotated_truth_image, c, d, (255, 0, 0),  10)

# # rotate our image by 45 degrees around the center of the image
M = cv2.getRotationMatrix2D((cx, cy), -angle, 1.0)
rotated_truth_image = cv2.warpAffine(rotated_truth_image, M, (rotated_truth_image.shape[1], rotated_truth_image.shape[0]))
cv2.namedWindow('rotated_truth_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('rotated_truth_image', rotated_truth_image.shape[1], rotated_truth_image.shape[0])
cv2.imshow('rotated_truth_image',rotated_truth_image)
cv2.waitKey(0)

cropped_truth_image = rotated_truth_image.copy()
y_offset = 0
cropped_truth_image = cropped_truth_image[min(cy, dy)- y_offset :max(cy, dy) + 1 + y_offset, 1200:(max(cx, dx) + d_to_c_error) + 1] 
cropped_truth_dims = cropped_truth_image.shape
cropped_truth_height = cropped_truth_dims[0]
print('cropped_truth_height:%d' %cropped_truth_height)
cropped_truth_width = cropped_truth_dims[1]
print('cropped_truth_width:%d' %cropped_truth_width)
# filtered_image = filtered_image & mask_rgb
cv2.namedWindow('cropped_truth_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('cropped_truth_image', cropped_truth_image.shape[1], cropped_truth_image.shape[0])
cv2.imshow('cropped_truth_image',cropped_truth_image)
cv2.waitKey(0)
cropped_truth_image = cropped_truth_image[::, 0:int(cropped_truth_image.shape[1] - MIN_HOLE_RADIUS_PIXELS) + 1] 
#cropped_image[::, int(max(cx, dx) - MIN_HOLE_RADIUS_PIXELS):max(cx, dx)]  = (0, 0, 0)
cv2.line(cropped_truth_image, c, d, (0, 0, 255),  1)
# filtered_image = filtered_image & mask_rgb
cv2.namedWindow('cropped_truth_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('cropped_truth_image', cropped_truth_image.shape[1], cropped_truth_image.shape[0])
cv2.imshow('cropped_truth_image',cropped_truth_image)
cv2.waitKey(0)

filtered_truth_image = cropped_truth_image.copy()

hsv = cv2.cvtColor(filtered_truth_image, cv2.COLOR_BGR2HSV)  
black = cv2.inRange(hsv, np.array([0, 0, 0]),  np.array([360,255, 100]))
cv2.namedWindow('black', cv2.WINDOW_NORMAL)
cv2.resizeWindow('black', black.shape[1], black.shape[0])
cv2.imshow('black',black)
cv2.waitKey(0)
kernel = np.ones((5, 5), np.uint8)
black = cv2.ximgproc.thinning(black)
cv2.namedWindow('black', cv2.WINDOW_NORMAL)
cv2.resizeWindow('black', black.shape[1], black.shape[0])
cv2.imshow('black',black)
cv2.waitKey(0)

skeletonized_truth_image = black.copy()
skeletonized_truth_image_dims = skeletonized_truth_image.shape
skeletonized_truth_image_height = skeletonized_truth_image_dims[0]
print('skeletonized_truth_image_height:%d' %skeletonized_truth_image_height)
skeletonized_truth_image_width = skeletonized_truth_image_dims[1]
print('skeletonized_truth_image_width:%d' %skeletonized_truth_image_width)

coords = []
xMillimeters = []
yMillimeters = []
for y in range(skeletonized_truth_image_height):
    for x in range(skeletonized_truth_image_width):
        if skeletonized_truth_image[y, x] == 255:
            coords.append((x, y))
            xMillimeter = (skeletonized_truth_image_width - x) * ratioMillimetersToPixels
            xMillimeters.append(xMillimeter)
            yMillimeter = (skeletonized_truth_image_height - y) * ratioMillimetersToPixels
            yMillimeters.append(yMillimeter)
            #cv2.circle(curve_image, (x, y), 2, (255, 0, 0), -1)
            break


DfMillimeters = pd.DataFrame(columns=['x', 'y'], data={'x':xMillimeters,'y':yMillimeters})
DfMillimeters.to_json('calib-observed.json')
points = np.array(coords)

pts = points.reshape(-1,1,2) # now shape [3,1,2]
# cv2.polylines(curve_image, [pts], False, (0, 0, 255), 5)
# cv2.imshow("curve_image",curve_image)
# cv2.waitKey(0)


final_image = cropped_truth_image.copy()
cv2.polylines(final_image, [pts], False, (255, 255, 255), 1)
cv2.namedWindow('final_image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('final_image', final_image.shape[1], final_image.shape[0])
cv2.imshow("final_image",final_image)
cv2.waitKey(0)
print(xMillimeters)
print(yMillimeters)
       

# contours, hierarchy = cv2.findContours(black, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# cv2.drawContours(filtered_truth_image, contours, -1, (255, 255, 255), cv2.FILLED) # this gives a binary mask 
# cv2.namedWindow('filtered_truth_image', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('filtered_truth_image', filtered_truth_image.shape[1], filtered_truth_image.shape[0])
# cv2.imshow('filtered_truth_image',filtered_truth_image)
# cv2.waitKey(0)

# gray_truth_image = cv2.cvtColor(cropped_truth_image, cv2.COLOR_BGR2GRAY)
# cv2.namedWindow('gray_truth_image', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('gray_truth_image', gray_truth_image.shape[1], gray_truth_image.shape[0])
# cv2.imshow('gray_truth_image',gray_truth_image)
# cv2.waitKey(0)
# ret, thresh = cv2.threshold(gray_truth_image,200,255,cv2.THRESH_BINARY_INV)
# cv2.namedWindow('thresh', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('thresh', thresh.shape[1], thresh.shape[0])
# cv2.imshow('thresh',thresh)
# cv2.waitKey(0)

# contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# cv2.drawContours(gray_truth_image, contours, -1, (255, 255, 255), cv2.FILLED) # this gives a binary mask 
# cv2.namedWindow('gray_truth_image', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('gray_truth_image', gray_truth_image.shape[1], gray_truth_image.shape[0])
# cv2.imshow('gray_truth_image',gray_truth_image)
# cv2.waitKey(0)


cv2.destroyAllWindows()